In [ ]:
# Required dependencies:
# numpy
# pandas (version 0.24 or later)
# shapely (interface to GEOS)
# fiona (interface to GDAL)
# pyproj (interface to PROJ; version 2.2.0 or later)

# Further, optional dependencies are:
# rtree (optional; spatial index to improve performance and required for overlay operations; interface to libspatialindex)
# psycopg2 (optional; for PostGIS connection)
# GeoAlchemy2 (optional; for writing to PostGIS)
# geopy (optional; for geocoding)

# For plotting, these additional packages may be used:
# matplotlib (>= 2.2.0)
# mapclassify (>= 2.2.0)

!pip install numpy
!pip install pandas
!pip install shapely
!pip install fiona
!pip install pyproj
!pip install geopandas

# Optional
!pip install rtree

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
%matplotlib inline

# Plot a shapefile

In [ ]:
import os

In [ ]:
# Shapefile extensions
# https://desktop.arcgis.com/en/arcmap/10.3/manage-data/shapefiles/shapefile-file-extensions.htm

In [ ]:
WORLD_MAP_PATH = os.path.join('.', 'Data', 'worldmap', 'World_Countries__Generalized_.shp')

In [ ]:
WORLD_MAP_PATH

In [ ]:
world_map_gdf = gpd.read_file(WORLD_MAP_PATH)
world_map_gdf.head()

In [ ]:
world_map_gdf.plot()

In [ ]:
# plot specific country by ISO code (filters just like pandas dataframe)
world_map_gdf[world_map_gdf.ISO=='US'].plot()

# Plot latitude and longitude data

In [ ]:
# Generate dummy data
from random import randrange
import pandas as pd
def generate_latlong(num=5):
    long = [randrange(-180,180) for i in range(num)]
    lat = [randrange(-90, 90) for i in range(num)]
    return lat, long

In [ ]:
long, lat = generate_latlong(5)
geo_data = pd.DataFrame()
geo_data['lat'] = lat
geo_data['long'] = long
geo_data['geometry'] = geo_data.apply(lambda x: Point(x.lat, x.long), axis=1)

In [ ]:
geo_data_gdf = gpd.GeoDataFrame(geo_data)
# geo_data_gdf.crs = {'init': 'epsg:4326'}
ax = world_map_gdf.plot()
geo_data_gdf.plot(ax=ax, color='r')

# Map formatting

In [ ]:
# color - fills inside
# outline - line
ax = world_map_gdf.plot(figsize=(20,20), color='grey', edgecolor='white', linewidth=0.25)
geo_data_gdf.plot(ax=ax, color='red', markersize=100, edgecolor='purple', linewidth=3, alpha=0.5)
ax.axis('off')

# Zoom area

In [ ]:
ax = world_map_gdf.plot(figsize=(20,20), color='grey', edgecolor='white', linewidth=0.25)
geo_data_gdf.plot(ax=ax, color='red', markersize=100, edgecolor='purple', linewidth=3, alpha=0.5)
ax.axis('off')
ax.set_xlim([-130, -60])
ax.set_ylim([20, 50])

# Spatial join

In [ ]:
long, lat = generate_latlong(20)
geo_data = pd.DataFrame()
geo_data['lat'] = lat
geo_data['long'] = long
geo_data['geometry'] = geo_data.apply(lambda x: Point(x.lat, x.long), axis=1)

In [ ]:
geo_data = gpd.GeoDataFrame(geo_data)

In [ ]:
ax = world_map_gdf.plot(figsize=(20,20))
geo_data.plot(ax=ax, color='r')

In [ ]:
# make sure crs match in order to spacial join properly
world_map_gdf.crs

In [ ]:
geo_data.crs = {'init': 'epsg:4326'}
# (world_map_gdf.crs, inplace=True)

In [ ]:
# the order of sjoin data matters. Keep all geo_data points that are inside of world_map_gdf geometry shapes
world_map_with_dot = gpd.sjoin(geo_data, world_map_gdf, how='inner', op='within')
world_map_with_dot.head()

In [ ]:
ax = world_map_gdf.plot(figsize=(20,20))
world_map_with_dot.plot(ax=ax, color='r')

In [ ]:
world_map_with_dot.COUNTRY.value_counts()